# Fine Tuning Transformer for Summary Generation

<a id='section03'></a>
### Fine Tuning the Model: Function

Here we define a training function that trains the model on the training dataset created above, specified number of times (EPOCH), An epoch defines how many times the complete data will be passed through the network.

This function is called in the `main()`

Following events happen in this function to fine tune the neural network:
- The epoch, tokenizer, model, device details, testing_ dataloader and optimizer are passed to the `train ()` when its called from the `main()`
- The dataloader passes data to the model based on the batch size.
- `language_model_labels` are calculated from the `target_ids` also, `source_id` and `attention_mask` are extracted.
- The model outputs first element gives the loss for the forward pass.
- Loss value is used to optimize the weights of the neurons in the network.
- After every 10 steps the loss value is logged in the wandb service. This log is then used to generate graphs for analysis. Such as [these](https://app.wandb.ai/abhimishra-91/transformers_tutorials_summarization?workspace=user-abhimishra-91)
- After every 500 steps the loss value is printed in the console.

In [ ]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we wnumerate over the training loader and passed to the defined network

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        labels = y[:, 1:].clone().detach()
        labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, decoder_input_ids=y_ids, labels=labels)
        loss = outputs[0]

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

<a id='section04'></a>
### Validating the Model Performance: Function

During the validation stage we pass the unseen data(Testing Dataset), trained model, tokenizer and device details to the function to perform the validation run. This step generates new summary for dataset that it has not seen during the training session.

This function is called in the `main()`

This unseen data is the 20% of `news_summary.csv` which was seperated during the Dataset creation stage.
During the validation stage the weights of the model are not updated. We use the generate method for generating new text for the summary.

It depends on the `Beam-Search coding` method developed for sequence generation for models with LM head.

The generated text and originally summary are decoded from tokens to text and returned to the `main()`

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask,
                max_length=150,
                num_beams=2,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True,
                )

            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

<a id='section05'></a>
### Main Function

The `main()` as the name suggests is the central location to execute all the functions/flows created above in the notebook. The following steps are executed in the `main()`:


<a id='section502'></a>
#### Importing and Pre-Processing the domain data

We will be working with the data and preparing it for fine tuning purposes.
*Assuming that the `news_summary.csv` is already downloaded in your `data` folder*

* The file is imported as a dataframe and give it the headers as per the documentation.
* Cleaning the file to remove the unwanted columns.
* A new string is added to the main article column `summarize: ` prior to the actual article. This is done because **T5** had similar formatting for the summarization dataset.
* The final Dataframe will be something like this:

|text|ctext|
|--|--|
|summary-1|summarize: article 1|
|summary-2|summarize: article 2|
|summary-3|summarize: article 3|

* Top 5 rows of the dataframe are printed on the console.

<a id='section503'></a>
#### Creation of Dataset and Dataloader

* The updated dataframe is divided into 80-20 ratio for test and validation.
* Both the data-frames are passed to the `CustomerDataset` class for tokenization of the new articles and their summaries.
* The tokenization is done using the length parameters passed to the class.
* Train and Validation parameters are defined and passed to the `pytorch Dataloader contstruct` to create `train` and `validation` data loaders.
* These dataloaders will be passed to `train()` and `validate()` respectively for training and validation action.
* The shape of datasets is printed in the console.


<a id='section504'></a>
#### Neural Network and Optimizer

* In this stage we define the model and optimizer that will be used for training and to update the weights of the network.
* We are using the `t5-base` transformer model for our project. You can read about the `T5 model` and its features above.
* We use the `T5ForConditionalGeneration.from_pretrained("t5-base")` commad to define our model. The `T5ForConditionalGeneration` adds a Language Model head to our `T5 model`. The Language Model head allows us to generate text based on the training of `T5 model`.
* We are using the `Adam` optimizer for our project. This has been a standard for all our tutorials and is something that can be changed updated to see how different optimizer perform with different learning rates.
* There is also a scope for doing more with Optimizer such a decay, momentum to dynamically update the Learning rate and other parameters. All those concepts have been kept out of scope for these tutorials.


<a id='section505'></a>
#### Training Model

* We call the `train()` with all the necessary parameters.
* Loss at every 500th step is printed on the console.


<a id='section506'></a>
#### Validation and generation of Summary

* After the training is completed, the validation step is initiated.
* As defined in the validation function, the model weights are not updated. We use the fine tuned model to generate new summaries based on the article text.
* An output is printed on the console giving a count of how many steps are complete after every 100th step.
* The original summary and generated summary are converted into a list and returned to the main function.
* Both the lists are used to create the final dataframe with 2 columns **Generated Summary** and **Actual Summary**
* The dataframe is saved as a csv file in the local drive.
* A qualitative analysis can be done with the Dataframe.

In [ ]:
def main():

    TRAIN_BATCH_SIZE = 4    # input batch size for training (default: 64) #dikecilin batch sizenya jadi 2/4
    VALID_BATCH_SIZE = 4    # input batch size for testing (default: 1000)
    TRAIN_EPOCHS = 20        # number of epochs to train (default: 10)
    VAL_EPOCHS = 20
    LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
    SEED = 42               # random seed (default: 42)
    MAX_LEN = 512
    SUMMARY_LEN = 150

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(SEED) # pytorch random seed
    np.random.seed(SEED) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained("t5-small")


    # Importing and Pre-Processing the domain data
    # Selecting the needed columns only.
    # Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task.
    df = pd.read_csv('/content/sample_data/news_summary.csv',encoding='latin-1')
    df = df[['text','ctext']]
    df.ctext = 'summarize: ' + df.ctext
    print(df.head())


    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest will be used for validation.
    train_size = 0.8
    train_dataset=df.sample(frac=train_size, random_state = SEED).reset_index(drop=True)
    val_dataset=df.drop(train_dataset.index).reset_index(drop=True)

    print("FULL Dataset: {}".format(df.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(val_dataset.shape))


    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)

    # Defining the parameters for creation of dataloaders
    train_params = {
        'batch_size': TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

    val_params = {
        'batch_size': VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)


    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained("t5-small")
    model = model.to(device)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

    # Training loop
    print('Initiating Fine-Tuning for the model on our dataset')

    for epoch in range(TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

    # Training loop
    print('Initiating Fine-Tuning for the model on our dataset')
    for epoch in range(TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

    # Save the model after training
    model.save_pretrained("/model")
    tokenizer.save_pretrained("/model")

    # Validation loop and saving the resulting file with predictions and acutals in a dataframe.
    # Saving the dataframe as predictions.csv
    print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
    for epoch in range(VAL_EPOCHS):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
        final_df.to_csv('predictions.csv')
        print('Output Files generated for review')

if __name__ == '__main__':
    main()

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


                                                text  \
0  The Administration of Union Territory Daman an...   
1  Malaika Arora slammed an Instagram user who tr...   
2  The Indira Gandhi Institute of Medical Science...   
3  Lashkar-e-Taiba's Kashmir commander Abu Dujana...   
4  Hotels in Maharashtra will train their staff t...   

                                               ctext  
0  summarize: The Daman and Diu administration on...  
1  summarize: From her special numbers to TV?appe...  
2  summarize: The Indira Gandhi Institute of Medi...  
3  summarize: Lashkar-e-Taiba's Kashmir commander...  
4  summarize: Hotels in Mumbai and other Indian c...  
FULL Dataset: (4514, 2)
TRAIN Dataset: (3611, 2)
TEST Dataset: (903, 2)


...

Completed 0
Completed 100
Completed 200
Output Files generated for review


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Completed 0
Completed 100
Completed 200
Output Files generated for review


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Completed 0
Completed 100
Completed 200
Output Files generated for review


In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge
import nltk
import pandas as pd

# Download NLTK data
nltk.download('punkt')

def calculate_rouge(predictions, actuals):
    # Join the tokenized words into strings
    references = [' '.join(reference) for reference in actuals]
    hypotheses = [' '.join(hypothesis) for hypothesis in predictions]

    rouge = Rouge()
    scores = rouge.get_scores(hypotheses, references, avg=True)

    rouge_scores = {
        'rouge-1': scores['rouge-1']['f'],
        'rouge-2': scores['rouge-2']['f'],
        'rouge-l': scores['rouge-l']['f'],
    }

    return rouge_scores

# Assuming you have loaded your predictions and actuals
predictions_df = pd.read_csv('predictions.csv')
predictions = predictions_df['Generated Text'].tolist()
actuals = predictions_df['Actual Text'].tolist()

# Tokenizing references and hypotheses
actuals = [nltk.word_tokenize(reference.lower()) for reference in actuals]
predictions = [nltk.word_tokenize(hypothesis.lower()) for hypothesis in predictions]

# Calculate Rouge scores
rouge_scores = calculate_rouge(predictions, actuals)

# Print Rouge scores
print("Rouge Scores:")
for metric, score in rouge_scores.items():
    print(f"{metric}: {score}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Rouge Scores:
rouge-1: 0.5863836602512773
rouge-2: 0.3976804501549606
rouge-l: 0.5507699581694352


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

# Path Google Drive
drive_path = '/content/gdrive/MyDrive/'

# Path file yang ingin disalin
source_path = '/model/tokenizer_config.json'

# Path tujuan di Google Drive
destination_path = '/content/drive/MyDrive/Model_v5'

# Salin file ke Google Drive
shutil.copy(source_path, destination_path)

'/content/drive/MyDrive/Model_v5/tokenizer_config.json'

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

def summarize_text(model, tokenizer, text):
    # Tokenize and generate summary
    input_ids = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    output = model.generate(input_ids, max_length=150, num_beams=2, length_penalty=2.0, early_stopping=True)
    generated_summary = tokenizer.decode(output[0], skip_special_tokens=True)

    # Count the number of tokens in the input text
    num_tokens_input = len(tokenizer.encode(text, max_length=512, truncation=True))

    return generated_summary, num_tokens_input

def main():
    # Load trained model and tokenizer
    model = T5ForConditionalGeneration.from_pretrained("/model")
    tokenizer = T5Tokenizer.from_pretrained("/model")

    # Sample news texts for summarization
    news_texts = [
        '''
        Telangana Congress chief Revanth Reddy is set to become the next chief minister of the southern state, as announced by the party on Tuesday. Congress general secretary KC Venugopal, in a press briefing, disclosed that the swearing-in ceremony is scheduled for December 7. Recognized for leading the party to a decisive triumph in the assembly elections, Reddy emerged as the face of Congress' successful campaign in Telangana. Following the announcement, Reddy conveyed his appreciation to Sonia Gandhi, Congress President Mallikarjun Kharge, Rahul Gandhi, and other prominent party leaders. In Mizoram, Zoram Peoples’ Movement (ZPM) President Lalduhoma is poised to become the next chief minister of Mizoram following his party's resounding victory in the recent state assembly elections. The ZPM, in the opposition, secured 27 seats out of 40, overshadowing the ruling Mizo National Front (MNF) which claimed 10 seats, relegating them to a distant second. The Bharatiya Janata Party (BJP) and Congress each secured 2 seats in the assembly. However, the Bharatiya Janata Party (BJP) is confronted with the crucial task of selecting potential chief ministerial candidates to lead new cabinets for the upcoming five years. In Rajasthan, BJP faces a challenging decision among Vasundhara Raje, Diya Kumari, and Baba Balaknath, or it might introduce an unexpected candidate. In Chhattisgarh, veteran leader Raman Singh is considered a strong contender for the chief ministerial role. Similarly, four-time Chief Minister of Madhya Pradesh, Shivraj Singh Chouhan, is likely to make a return for his fifth term.
        '''
    ]

    # Summarize each news text
    for idx, news_text in enumerate(news_texts, start=1):
        generated_summary, num_tokens_input = summarize_text(model, tokenizer, news_text)
        print(f"\nNews {idx}:\n")
        print(f"Original Text (Number of Tokens): {num_tokens_input}\n{news_text}\n")
        print(f"Generated Summary:\n{generated_summary}\n")

if __name__ == '__main__':
    main()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



News 1:

Original Text (Number of Tokens): 397

        Telangana Congress chief Revanth Reddy is set to become the next chief minister of the southern state, as announced by the party on Tuesday. Congress general secretary KC Venugopal, in a press briefing, disclosed that the swearing-in ceremony is scheduled for December 7. Recognized for leading the party to a decisive triumph in the assembly elections, Reddy emerged as the face of Congress' successful campaign in Telangana. Following the announcement, Reddy conveyed his appreciation to Sonia Gandhi, Congress President Mallikarjun Kharge, Rahul Gandhi, and other prominent party leaders. In Mizoram, Zoram Peoples’ Movement (ZPM) President Lalduhoma is poised to become the next chief minister of Mizoram following his party's resounding victory in the recent state assembly elections. The ZPM, in the opposition, secured 27 seats out of 40, overshadowing the ruling Mizo National Front (MNF) which claimed 10 seats, relegating them to a d

# Contoh Summary

**1. Original Text :**

Telangana Congress chief Revanth Reddy is set to become the next chief minister of the southern state, as announced by the party on Tuesday. Congress general secretary KC Venugopal, in a press briefing, disclosed that the swearing-in ceremony is scheduled for December 7. Recognized for leading the party to a decisive triumph in the assembly elections, Reddy emerged as the face of Congress' successful campaign in Telangana. Following the announcement, Reddy conveyed his appreciation to Sonia Gandhi, Congress President Mallikarjun Kharge, Rahul Gandhi, and other prominent party leaders.  In Mizoram, Zoram Peoples’ Movement (ZPM) President Lalduhoma is poised to become the next chief minister of Mizoram following his party's resounding victory in the recent state assembly elections. The ZPM, in the opposition, secured 27 seats out of 40, overshadowing the ruling Mizo National Front (MNF) which claimed 10 seats, relegating them to a distant second. The Bharatiya Janata Party (BJP) and Congress each secured 2 seats in the assembly. However, the Bharatiya Janata Party (BJP) is confronted with the crucial task of selecting potential chief ministerial candidates to lead new cabinets for the upcoming five years. In Rajasthan, BJP faces a challenging decision among Vasundhara Raje, Diya Kumari, and Baba Balaknath, or it might introduce an unexpected candidate. In Chhattisgarh, veteran leader Raman Singh is considered a strong contender for the chief ministerial role. Similarly, four-time Chief Minister of Madhya Pradesh, Shivraj Singh Chouhan, is likely to make a return for his fifth term.


**Summary:**

Earlier, Congress General Secretary KC Venugopal announced that Telangana Congress chief Revanth Reddy would be the next Chief Minister of the state. The ceremony, scheduled for December 7, will be held to mark the assumption of office of the state's Chief Minister. Reportedly, Revanth had led the party to a decisive victory in the recent state Assembly elections.



**2. Original Text:**

Taylor Swift has capped off a stellar 2023 by being named Time Magazine's person of the year. The star, whose Eras tour broke box office records and provoked an inquiry into Ticketmaster's sales practices, follows the likes of Barack Obama, Greta Thunberg and Volodymyr Zelensky. She told the magazine that she is "the proudest and happiest I've ever felt". The award goes to an event or person deemed to have had the most influence on global events over the past year. The singer also admitted to the magazine that the toll of her 180-minute Eras concerts often left her feeling physically exhausted. After a run of shows, "I do not leave my bed except to get food and take it back to my bed and eat it there," she said. "I can barely speak because I've been singing for three shows straight. Every time I take a step my feet go crunch, crunch, crunch from dancing in heels." The star also talked about her blossoming romance with American Football star Travis Kelce. The couple hit the headlines in September when Swift was spotted at with Kelce's mother at one of his games. "By the time I went to that first game, we were a couple," she explained, adding they had first hooked up over the summer. "I think some people think that they saw our first date at that game? We would never be psychotic enough to hard-launch a first date." But Swift's love life is small beans compared to her cultural impact. Already a superstar before 2023, her career has reached new heights thanks to the Eras tour - which sees the singer perform a career-spanning 45-song set every night. Demand for tickets was so high that it crashed Ticketmaster's website, prompting a hearing into its business practices by the US Senate. When the tour began in the summer, ticketless fans gathered in car parks just to hear the music. In Seattle, her concerts generated seismic activity equivalent to a 2.3 magnitude earthquake. "It feels like the breakthrough moment of my career, happening at 33," she told Time. "And for the first time in my life, I was mentally tough enough to take what comes with that." Swift's imperial phase comes after a period where she was vilified for her positions on feminism and politics - although her silence on those issues stemmed from nothing more sinister than a lack of confidence. After speaking out against Donald Trump and in favour of abortion rights, she hit a creative purple patch with the pandemic-era albums Folklore and Evermore. Showcasing a more organic, indie-folk approach than the country-pop that made her famous, the records confirmed her status as a generational songwriting talent. She cemented her comeback with last year's Midnights - a bleary, sleep-deprived collection of pop songs based around the thoughts that keep her up at night.

**Summarize:**

'Eras' singer Taylor Swift has been named Time Magazine's person of the year, 2023. Swift, who sang for three consecutive shows, said that she can barely speak because she has been singing for three nights straight. She added, "Every time I take a step my feet go crunch, crunch, crunch from dancing in heels."


**3. Original Text :**

The controversy arose after two “racist” royals were named in the Dutch edition of Omid Scobie’s new book, Endgame
A royal author has claimed that Meghan Markle could be dropped by her talent agency because of Omid Scobie’s new controversial book, Endgame: Inside the Royal Family and the Monarchy’s Fight for Survival. Angela Levin, while speaking to GB News, said that Patrick Christys that William Morris Endeavour is “horrified” imagining what consequences the Sussex brand could have as a result of the book.
Angela said that the controversy around the race row can affect Meghan, and she might as well be dropped by the agency. “They said they were ‘horrified’, that can’t just be about the two names that were mentioned, it must be about their client, Meghan,” she said. “It seems to me that they will think very carefully what they do next. I have never heard such a well known agency be clear with how they Angela added, "Meghan needs to be very careful. A lot of people have dropped her. The agency have told her to ‘stop moaning’, because people are ‘getting fed up’ with it. If they know feel they have an even harder job to do in order to get her famous, it’s going to be difficult.”
On being asked if Meghan may be dropped by the talent agency, Angela responded, “Yes.” “It is going to be very difficult to get the glamour she wants”, she said.
The controversy arose after two “racist” royals were named in the Dutch edition of Omid Scobie’s new book. The book was released on Tuesday, November 28. Later, the book was removed from the shelves across the Netherlands. It will reportedly be back on the shelves again on December 8.
Back in March 2021, Meghan, who is biracial, told Oprah Winfrey during an interview that a member of the family had said something about “how dark” Archie’s skin would be when he was born. Archie was born in May 2019.
        

**Summary :**

the royal author has claimed that Meghan Markle could be dropped by her talent agency because of Omid Scobie's new book. Angela Levin said, "The agency have told her to stop moaning, because people are ‘getting fed up’ with it. If they know feel they have an even harder job to do in order to get her famous, it's going to be difficult."
